In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from scipy.stats import skew, kurtosis

C:\Users\jana\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Defining all my functions here

In [2]:
def calculate_moments(all_images): #takes in array of vector images
    momentsArray = []
    for image in all_images:
        # Calculate mean
        mean = np.mean(image)

        # Calculate standard deviation
        std_dev = np.std(image)

        # Calculate skewness
        skewness = skew(image.reshape(-1))  # Reshape to 1D array for skew function

        # Calculate kurtosis
        kurt = kurtosis(image.reshape(-1))  # Reshape to 1D array for kurtosis function
        
        momentsArray.append([mean, std_dev, skewness, kurt])
    return np.array(momentsArray)

def calculate_hu_moments(all_images): #takes in vector
    momentsArray = []
    for image_array in all_images:
        cv2.imwrite('myImage.png', image_array) #convert vector to image
        image = cv2.imread('myImage.png', cv2.IMREAD_GRAYSCALE) #read in image as grayscale (needs to be grayscale for moments function)
        _,image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY) 
        
        #calculate moments
        moments = cv2.moments(image) 
        
        # Calculate Hu Moments
        huMoments = cv2.HuMoments(moments)
        momentsArray.append(huMoments)
        
    return np.array(momentsArray)

def combine_inputs(all_images): #takes in array of vector images; outputs 2D vector (one 1D array dedicated to each image)
    #moments
    moments_array = calculate_moments(all_images) #np array of stats for each image
    
    #hu moments
    hu_moments_array = calculate_hu_moments(all_images) #np array of hu moments for each image

    #flatten arrays
    all_images_flat = tf.keras.layers.Flatten()(all_images)
    hu_moments_flat = tf.keras.layers.Flatten()(hu_moments_array)

    return np.concatenate((all_images_flat, hu_moments_flat, moments_array), axis=-1)

For initial development purposes, I'm using the mnist dataset of handdrawn digits.
This should be removed for actual project

In [3]:
mnist = tf.keras.datasets.mnist #collect data

#split data into training and testing data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
#defining inputs with mnist data
x_train_with_moments = combine_inputs(x_train)
x_test_with_moments = combine_inputs(x_test)

#normalise data
x_train_with_moments = tf.keras.utils.normalize(x_train_with_moments, axis = 1)
x_test_with_moments = tf.keras.utils.normalize(x_test_with_moments, axis = 1)

KeyboardInterrupt: 

For initial development purposes, this is the NN I've been using. Not at all optimised for this application.

In [ ]:
model = tf.keras.models.Sequential()
#model.add(tf.keras.layers.Flatten(input_shape=(28, 28))) - only needs to be used if just using mnist data and not extra info
model.add(tf.keras.layers.Dense(128, activation = 'relu'))  #add dense layer, where each neuron is connected to each neuron of next layer
model.add(tf.keras.layers.Dense(128, activation = 'relu')) 
model.add(tf.keras.layers.Dense(128, activation = 'relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax')) #each unit represents number (0-9); softmax ensures all outputs add up to 1 (essentially outputs are percentages)
                                                            
#compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = ['accuracy'])

#train the model
model.fit(x_train, y_train, epochs = 5)

: 

Test model with test data

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test)

print(loss)
print(accuracy)

: 